In [49]:
# import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import tifffile as tiff
import seaborn as sns
import pickle

In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")
 
if module_path not in sys.path:
    sys.path.append(module_path)

In [52]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

df_meta_path = data_dir / 'specificity_PLA' / 'metadata' / 'imgs_reg.csv'
df_imgs = pd.read_csv(df_meta_path)


In [53]:
df_imgs

,Condition,FOV,Path
0,Control,Control,y:\coskun-lab\Thomas\23_PLA_revision\data\spec...
1,KO,KO,y:\coskun-lab\Thomas\23_PLA_revision\data\spec...


In [54]:
# df_imgs['Condition'] = ['KO', 'Control']
# df_imgs['FOV'] = ['KO', 'Control']

# Plot examples

Revision Figure on tratement 
Need to look at FOV 6 and 7 and compared control and treated

In [109]:
import napari 
from sklearn.neighbors import NearestNeighbors
import scipy 

# remove first and last consecutive zeros along dimension
def removeFirstAndLast(zeroArray):
    
    # remove first
    for ii, val in enumerate(zeroArray):
        if np.sum(zeroArray[:ii + 1]) != ii + 1: # all consecutive trues
            break # exit loop
            
        else:
            lowest = ii # index of last of the first consecutive zeros
            
    # remove last
    for ii, val in enumerate(np.flip(zeroArray)):
        if np.sum(zeroArray[:ii + 1]) != ii + 1: # all consecutive trues
            break # exit loop
            
        else:
            highest = len(zeroArray) - ii # index of first of the last consecutive zeros            
    
    return (lowest, highest)

# remove black borders from napari screenshot (2D image)
def removeBorders(RGB):

    rowMin = []
    rowMax = []
    colMin = []
    colMax = []
    for jj in range(3): # each RGB
        
        img = RGB[:, :, jj]
        # find first and last rows and cols of all zeros
        # remove any rows of zeros
        zeroRows = np.all(img == 0, axis = 1)
        zeroCols = np.all(img == 0, axis = 0)

        posRows = removeFirstAndLast(zeroRows)
        posCols = removeFirstAndLast(zeroCols)            
        
        rowMin.append(posRows[0])
        rowMax.append(posRows[1])
        colMin.append(posCols[0])
        colMax.append(posCols[1])
        
    # make sure border signal is not cut out
    rowMin = np.min(rowMin)
    rowMax = np.max(rowMax)
    colMin = np.min(colMin)
    colMax = np.max(colMax)
        
    cropped = RGB[rowMin: rowMax, colMin: colMax, :] 
    
    return cropped

def get_NN_radius(data, r):
    fit = NearestNeighbors(radius=r).fit(data)
    m = fit.radius_neighbors(data, return_distance=True, sort_results=True)

    # Put in dataframe format
    neighbours = pd.DataFrame(m[1].tolist(), index = data.index)
    
    A = fit.radius_neighbors_graph(data)
    return neighbours, A

def get_loc_ppi(df, Condition, FOV, x_min, y_min, x_max, y_max, PPI_name, merge=False, r=5):
    df_ppi = df[(df.Condition == Condition) & (df.FOV == FOV)]
    df_ppi = df_ppi[df_ppi.PPI == PPI_name]
    df_ppi = df_ppi[(df_ppi.column > x_min) & (df_ppi.column < x_max) 
                    & (df_ppi.row > y_min) & (df_ppi.row < y_max)] 
    if merge:
         # Get dot neighboring graph with user defined radius
        nei, A = get_NN_radius(df_ppi[['row', 'column']], r=r)

        # Assign new labels based on connected components label
        labels = scipy.sparse.csgraph.connected_components(A, directed=False)[1]
        df_ppi['CC_label'] = labels

        # Group by CC and extract mean position
        df_ppi = df_ppi.groupby(['CC_label'])[['row', 'column']].mean().astype(np.uint32)
        
    points = df_ppi[['row', 'column']].to_numpy()
    points = points - np.array([y_min, x_min])
    return points



In [110]:
# Read PPI 
PPI_save_path =  data_dir /'specificity_PLA' / 'PPI'

dfs = []
for path in os.listdir(PPI_save_path):
    if 'csv' in path:
        df = pd.read_csv(PPI_save_path / path)
        dfs.append(df)

df = pd.concat(dfs)
df = df.rename(columns={'x': 'row', 'y': 'column'})
df.head()

,Cell,Nuclei,column,row,PPI,Condition,FOV
0,178,173,2708,3356,Sox2/Oct4,Control,Control
1,0,0,3466,430,Sox2/Oct4,Control,Control
2,13,0,249,1464,Sox2/Oct4,Control,Control
3,0,0,1334,912,Sox2/Oct4,Control,Control
4,103,101,1616,1340,Sox2/Oct4,Control,Control


In [111]:
image_dir = (Path().cwd().parents[0] / 'figures').absolute() / 'PLA_scatter'
image_dir.mkdir(parents=True, exist_ok=True)

In [112]:
PPI_names = df.PPI.unique()
PPI_names

array(['Sox2/Oct4', 'Bim/Tom20'], dtype=object)

## B08

In [67]:
df_imgs

,Condition,FOV,Path
0,Control,Control,y:\coskun-lab\Thomas\23_PLA_revision\data\spec...
1,KO,KO,y:\coskun-lab\Thomas\23_PLA_revision\data\spec...


In [68]:
# Get images
index = 0
row = df_imgs.iloc[index]
print(row)
path = row.Path

with h5py.File(path, 'r') as f:
    imgs = f['imgs'][:]
    markers = f['imgs'].attrs['Marker']


Condition                                              Control
FOV                                                    Control
Path         y:\coskun-lab\Thomas\23_PLA_revision\data\spec...
Name: 0, dtype: object


In [69]:
# Define matching bbox to figures
x_min, y_min = 3636, 2314
x_max, y_max = 3868, 2500
DAPI_ch = 0

# Get DAPI image
dapiImg = imgs[0, y_min:y_max, x_min:x_max]

# Get ppi location
points = [get_loc_ppi(df, row.Condition, row.FOV, x_min, y_min, x_max, y_max, PPI_name) for PPI_name in PPI_names]


In [70]:
# Napari viewer
viewer = napari.view_image(dapiImg, colormap='blue')
points_layer = viewer.add_points(points[0], size=8,face_color='magenta',)
points_layer = viewer.add_points(points[1], size=8,face_color='lime',)

# Get screenshot
screenshot = viewer.screenshot(size=np.array(dapiImg.shape)*2)
viewer.close()
# Save image
tiff.imsave(image_dir / f'specificity_Control_1.tif', removeBorders(screenshot))

C:\Users\thu71\AppData\Local\Temp\ipykernel_4060\1279383561.py:10: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(image_dir / f'specificity_Control_1.tif', removeBorders(screenshot))


In [71]:
# Napari viewer
viewer = napari.view_image(imgs[:, y_min:y_max, x_min:x_max])

## B06

In [113]:
# Get images
index = 1
row = df_imgs.iloc[index]
print(row)
path = row.Path

with h5py.File(path, 'r') as f:
    imgs = f['imgs'][:]
    markers = f['imgs'].attrs['Marker']


Condition                                                   KO
FOV                                                         KO
Path         y:\coskun-lab\Thomas\23_PLA_revision\data\spec...
Name: 1, dtype: object


In [114]:
# Define matching bbox to figures
x_min, y_min = 1752, 2892
x_max, y_max = 1984, 3064
DAPI_ch = 0

# Get DAPI image
dapiImg = imgs[0, y_min:y_max, x_min:x_max]

# Get ppi location
points = [get_loc_ppi(df, row.Condition, row.FOV, x_min, y_min, x_max, y_max, PPI_name, merge=False, r = 7.5) for PPI_name in PPI_names]


In [116]:
# Napari viewer
viewer = napari.view_image(dapiImg, colormap='blue')
points_layer = viewer.add_points(points[0], size=8,face_color='magenta',)
points_layer = viewer.add_points(points[1], size=8,face_color='lime',)

# Get screenshot
screenshot = viewer.screenshot(size=np.array(dapiImg.shape)*2)
viewer.close()
# Save image
tiff.imsave(image_dir / f'specificity_KO_1.tif', removeBorders(screenshot))

C:\Users\thu71\AppData\Local\Temp\ipykernel_4060\2308372758.py:10: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(image_dir / f'specificity_KO_1.tif', removeBorders(screenshot))
